In [1]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
df_distance = pd.read_excel("F:/BI/第十三周/L21/distance.xlsx", index_col=0)

In [4]:
df_distance

,北京,天津,哈尔滨,长春,沈阳,石家庄,呼和浩特,太原,上海,杭州,...,成都,重庆,南宁,贵阳,昆明,武汉,长沙,南昌,香港,澳门
北京,0,122476,1229869,987941,682963,294125,491658,487353,1210971,1277042,...,1835076,1753571,2346302,2086066,2587861,1161356,1477547,1431143,2203897,2273144
天津,122476,0,1201742,959814,654836,314982,614846,508210,1093989,1217026,...,1855933,1774428,2343851,2083615,2585410,1158905,1475096,1358857,2201446,2270693
哈尔滨,1229869,1201742,0,244488,551085,1493810,1684809,1687038,2269863,2392900,...,3034761,2953256,3533141,3272905,3774700,2348195,2664386,2534550,3390736,3459983
长春,987941,959814,244488,0,310181,1252906,1431315,1446134,2028959,2151996,...,2793857,2712352,3292237,3032001,3533796,2107291,2423482,2293646,3149832,3219079
沈阳,682963,654836,551085,310181,0,950093,1171538,1143321,1726146,1849183,...,2491044,2409539,2989424,2729188,3230983,1804478,2120669,1990833,2847019,2916266
石家庄,294125,314982,1493810,1252906,950093,0,594851,214615,1170170,1180721,...,1551896,1470391,2086876,1826211,2260044,909276,1218121,1243653,1974526,2013718
呼和浩特,491658,614846,1684809,1431315,1171538,594851,0,443246,1703117,1724699,...,1728512,1664862,2529565,2046276,2436660,1393252,1681446,1738006,2458502,2463568
太原,487353,508210,1687038,1446134,1143321,214615,443246,0,1365196,1357544,...,1371477,1289972,2094228,1671386,2079625,957915,1246109,1302669,2023165,2028231
上海,1210971,1093989,2269863,2028959,1726146,1170170,1703117,1365196,0,179942,...,1956341,1702128,1904381,1816148,2317943,835747,1081808,700706,1493623,1563775
杭州,1277042,1217026,2392900,2151996,1849183,1180721,1724699,1357544,179942,0,...,1885005,1621852,1720463,1669561,2183377,755471,889395,515972,1309705,1379857


In [5]:
class VRP:
    def __init__(self, df_distance, num_vehicles=1, depot=0, city_names=None):
        # 设置城市名称
        self.df = df_distance
        self.num_vehicles = num_vehicles
        self.depot = depot
        self.all_city = self.df.columns
        if city_names is not None:
            self.city_names = city_names
            self.df = self.df.loc[self.all_city.isin(city_names),
                                  self.city_names]
        else:
            self.city_names = self.all_city

    def get_solution(self, manager, routing, solution):
        # 记录每辆车的里程
        distance_list = []
        # 记录每辆车的路径规划
        route_list = []

        for vehicle_id in range(self.num_vehicles):
            route = []
            # 从vehicle_id的起始节点出发
            index = routing.Start(vehicle_id)
            # 记录车最终行驶的距离
            route_distance = 0
            # routing.IsEnd 判断路径是否结束
            while not routing.IsEnd(index):
                # IndexToNode将manager中的index转换为distance_matrix中的index
                name_index = manager.IndexToNode(index)
                route.append(name_index)
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                # 针对vehicle=0，统计从previous_index到index节点的距离
                route_distance += routing.GetArcCostForVehicle(
                    previous_index, index, 0)
            route_list.append(route)
            distance_list.append(route_distance)
        return route_list, distance_list

    def create_data_model(self):
        data = {}
        data['distance_matrix'] = self.df.values / 1000  # 转公里
        data['num_vehicles'] = self.num_vehicles
        data['depot'] = self.depot
        return data

    def work(self):
        # step1，初始化数据，得到3个参数的字典
        data = self.create_data_model()

        # 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'],
                                               data['depot'])

        # step2，创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # step3，计算两点之间的距离，输入2个节点的index，输出2个节点之间的距离
        def distance_callback(from_index, to_index):
            # 将index转为distance_matrix中的index
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        # step4，注册距离函数
        transit_callback_index = routing.RegisterTransitCallback(
            distance_callback)

        # Define cost of each arc. 定义每个边的成本(arc 指边)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # step5，添加距离约束
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack 不松弛
            10000,  # 车辆最大行驶距离
            True,  # start cumul to zero 开始累计到零
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)

        # 尽量减少车辆之间的最大距离 全局差值成本系数100，值大则车辆间距离小；值小，则车辆间距离大；
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        # step6，设置参数策略，Setting first solution heuristic. 设置第一个启发式搜索的策略
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        # PATH_CHEAPEST_ARC 指边之间路径最短的策略
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # step7，求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # step8，输出结果
        route_list, distance_list = self.get_solution(manager, routing,
                                                      solution)
        return route_list, distance_list


if __name__ == "__main__":
    model = VRP(df_distance, 4, 0)
    route_list, distance_list = model.work()
    print(route_list)
    print(distance_list)

[[0, 7, 18, 19, 17, 22, 6], [0, 21], [0, 5, 20, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 13, 16], [0, 14, 28, 29, 10, 9, 8, 15, 1, 4, 2, 3]]
[6341, 7096, 6749, 6845]


In [6]:
sum(distance_list)

27031

In [10]:
# 车辆路线
for i, route in enumerate(route_list):
    print(f"第{i + 1}辆车行驶路线为:")
    print(f"由{df_distance.index[route[0]]}出发: ->", end=' ')
    for j in route[1:]:
        if route.index(j) < len(route) - 1:
            print(f"经：{df_distance.index[j]} ->", end=' ')
        else:
            print(f"到：{df_distance.index[j]} 结束")
            print(f"行驶里程为：{distance_list[i]}")
    print("-" * 135)

第1辆车行驶路线为:
由北京出发: -> 经：太原 -> 经：银川 -> 经：西宁 -> 经：兰州 -> 经：乌鲁木齐 -> 到：呼和浩特 结束
行驶里程为：6341
---------------------------------------------------------------------------------------------------------------------------------------
第2辆车行驶路线为:
由北京出发: -> 到：拉萨 结束
行驶里程为：7096
---------------------------------------------------------------------------------------------------------------------------------------
第3辆车行驶路线为:
由北京出发: -> 经：石家庄 -> 经：西安 -> 经：成都 -> 经：重庆 -> 经：贵阳 -> 经：昆明 -> 经：南宁 -> 经：海口 -> 经：澳门 -> 经：广州 -> 经：香港 -> 经：南昌 -> 经：合肥 -> 到：济南 结束
行驶里程为：6749
---------------------------------------------------------------------------------------------------------------------------------------
第4辆车行驶路线为:
由北京出发: -> 经：郑州 -> 经：武汉 -> 经：长沙 -> 经：福州 -> 经：杭州 -> 经：上海 -> 经：南京 -> 经：天津 -> 经：沈阳 -> 经：哈尔滨 -> 到：长春 结束
行驶里程为：6845
---------------------------------------------------------------------------------------------------------------------------------------
